In [43]:
import getNewReading as gnr
import kinneretDrawGraph as kdg
import pandas as pd
import math

In [41]:
df = gnr.importReadings()
df

,level,1day,7day,1month
date,,,,
2020-12-24,-209.885,0.005,0.020,0.100
2020-12-23,-209.890,0.010,0.050,0.100
2020-12-22,-209.895,0.010,0.080,0.095
2020-12-21,-209.895,0.015,0.090,0.105
2020-12-20,-209.900,0.015,0.090,0.115
...,...,...,...,...
1966-11-01,-210.390,0.505,0.505,0.385
1966-10-15,-210.320,0.435,0.435,0.245
1966-10-01,-210.270,0.385,0.385,0.135


In [38]:
# Delete all the calculated values
df.drop(columns=['1day','7day','1month'],inplace=True)
df

KeyError: "['1day' '7day' '1month'] not found in axis"

In [55]:

ind = df.index[0]

for i in range(10):
    dateOff = pd.DateOffset(days=-i)
    delta = kdg.getLevelDelta(df,ind,dateOff)
    lev = df.iloc[0]['level']
    day7 = df.iloc[0]['7day']
    

    print(f'{i:2} - {lev:.3f} - {delta:.3f} - {day7:.3f}')

 0 - -209.885 - 0.000 - 0.080
 1 - -209.885 - 0.005 - 0.080
 2 - -209.885 - 0.010 - 0.080
 3 - -209.885 - 0.010 - 0.080
 4 - -209.885 - 0.015 - 0.080
 5 - -209.885 - 0.015 - 0.080
 6 - -209.885 - 0.020 - 0.080
 7 - -209.885 - 0.020 - 0.080
 8 - -209.885 - 0.050 - 0.080
 9 - -209.885 - 0.080 - 0.080


In [5]:
df = gnr.updateCalcValues(df)
df

,level,1day,7day,1month
date,,,,
2020-12-24,-209.885,0.005,0.020,0.100
2020-12-23,-209.890,0.010,0.050,0.100
2020-12-22,-209.895,0.010,0.080,0.095
2020-12-21,-209.895,0.015,0.090,0.105
2020-12-20,-209.900,0.015,0.090,0.115
...,...,...,...,...
1966-11-01,-210.390,0.505,0.505,0.385
1966-10-15,-210.320,0.435,0.435,0.245
1966-10-01,-210.270,0.385,0.385,0.135


In [45]:
def naCheckDelta(df,x,y,dateOff,reCalc = False):
    """ Returns the delta value for this offset first checking if the y value is NAN
        If reCalc is True for a recalc """
    if reCalc OR math.isnan(y):
        return(kdg.getLevelDelta(df,x,dateOff))
    else:
        return y


def updateCalcValues(df,reCalc = False):
    """ Updates the 1day 7day and 1month calc values in the Dataframe
        Only updating what has changed. """
        
    dateOff1m = pd.DateOffset(months=-1)
    dateOff7d = pd.DateOffset(days=-7)
    dateOff1d = pd.DateOffset(days=-1)
    
    df['1day'] = [naCheckDelta(df,x,y,dateOff1d,reCalc) for x,y in zip(df.index,df['1day'])]
    df['7day'] = [naCheckDelta(df,x,y,dateOff7d,reCalc) for x,y in zip(df.index,df['7day'])]
    df['1month'] = [naCheckDelta(df,x,y,dateOff1m,reCalc) for x,y in zip(df.index,df['1month'])]
    return df

df = updateCalcValues(df,reCalc==True)


,level,1day,7day,1month
date,,,,
2020-12-24,-209.885,0.005,0.020,0.100
2020-12-23,-209.890,0.010,0.050,0.100
2020-12-22,-209.895,0.010,0.080,0.095
2020-12-21,-209.895,0.015,0.090,0.105
2020-12-20,-209.900,0.015,0.090,0.115
...,...,...,...,...
1966-11-01,-210.390,0.505,0.505,0.385
1966-10-15,-210.320,0.435,0.435,0.245
1966-10-01,-210.270,0.385,0.385,0.135


In [57]:
upsampled = df.resample('1D')
interpolated = upsampled.interpolate(method='linear')

In [58]:
interpolated

,level,1day,7day,1month
date,,,,
1966-09-01,-210.020000,0.135000,0.135000,0.135
1966-09-02,-210.027857,0.142857,0.142857,0.135
1966-09-03,-210.035714,0.150714,0.150714,0.135
1966-09-04,-210.043571,0.158571,0.158571,0.135
1966-09-05,-210.051429,0.166429,0.166429,0.135
...,...,...,...,...
2020-12-20,-209.900000,0.015000,0.090000,0.115
2020-12-21,-209.895000,0.015000,0.090000,0.105
2020-12-22,-209.895000,0.010000,0.080000,0.095


In [52]:
dataFile = 'data/levels-pd-calc2.csv'
colHead = ['level','1day','7day','1month']
df.round({'1day': 3,'7day': 3, '1month': 3}).to_csv(dataFile, index_label='date', columns=colHead)

In [30]:
dateOff = pd.DateOffset(days=-1)
ind = df.index[5]
# df['1dayT'] = [gnr.naCheckDelta(df,x,y,dateOff1d) for x,y in zip(df.index,df['1day'])]
# df['gap'] = [x for x in df.]
timeAgo = dateOff + ind
filt = df.index.get_loc(timeAgo, method='nearest')
oldLevel = df.iloc[filt]

print(oldLevel)

level    -209.935
1day        0.080
7day        0.075
1month      0.105
1dayT       0.080
Name: 2020-12-16 00:00:00, dtype: float64
